# Exercise 1: Write code to implement the convolution efficiently


### Big Picture
Imagine you have a collection of photos, and you want to find specific patterns in them - like edges, textures, or simple shapes. This notebook shows three different ways to do this "pattern finding" (convolution), kind of like having three different tools to search for hidden patterns in pictures.

In [4]:
import numpy as np
import time


In [5]:

# Load debugging data
debug_file = '../data/debug_info.npz'
load_data = np.load(debug_file)


In [6]:
# Extract data
X = load_data['X']  # shape: 3072 × n with n = 5
Fs = load_data['Fs']  # shape: f × f × 3 × nf, with f = 4, nf = 2

In [7]:
# Get dimensions
f = Fs.shape[0]  # filter size
nf = Fs.shape[3]  # number of filters
n = X.shape[1]  # number of images
n_p = (32 // f) ** 2  # number of patches per image

# Reshape and transpose X to get images in the right format
X_ims = np.transpose(X.reshape((32, 32, 3, n), order='F'), (1, 0, 2, 3))


In [8]:
# Print shapes of all arrays and values
print("f:", f)
print("nf:", nf)
print("n:", n)
print("n_p:", n_p)
print("X shape:", X.shape)
print("Fs shape:", Fs.shape) 
print("X_ims shape:", X_ims.shape)


f: 4
nf: 2
n: 5
n_p: 64
X shape: (3072, 5)
Fs shape: (4, 4, 3, 2)
X_ims shape: (32, 32, 3, 5)


In [9]:
conv_outputs = np.zeros((n_p, nf, n))

# Loop through each image
for i in range(n):
    # Loop through each filter
    for j in range(nf):
        patch_idx = 0
        # Loop through patches in the image
        for h in range(0, 32, f):
            for w in range(0, 32, f):
                # Extract patch
                patch = X_ims[h:h+f, w:w+f, :, i]
                # Get filter
                filter = Fs[:, :, :, j]
                # Compute dot product
                result = np.multiply(patch, filter).sum()
                conv_outputs[patch_idx, j, i] = result
                patch_idx += 1

In [10]:
conv_outputs_reshaped = conv_outputs.reshape((8, 8, nf, n))


In [11]:

# Compare with provided outputs
expected_outputs = load_data['conv_outputs']
print("Max difference:", np.abs(conv_outputs_reshaped - expected_outputs).max())

Max difference: 0.0


## Step 2 - Matrix Multiplication

In [12]:
# Initialize MX matrix of shape (n_p, f * f * 3, n)
MX = np.zeros((n_p, f * f * 3, n))

# Fill MX with patches
for i in range(n):
    patch_idx = 0
    for h in range(0, 32, f):
        for w in range(0, 32, f):
            # Extract patch and reshape it to a row vector
            patch = X_ims[h:h+f, w:w+f, :, i]
            MX[patch_idx, :, i] = patch.reshape((1, f * f * 3), order='C')
            patch_idx += 1

# Flatten the filters
Fs_flat = Fs.reshape((f * f * 3, nf), order='C')

# Initialize output array for matrix multiplication version
conv_outputs_mat = np.zeros((n_p, nf, n))

# Compute convolution using matrix multiplication
for i in range(n):
    conv_outputs_mat[:, :, i] = np.matmul(MX[:, :, i], Fs_flat)

# Compare with previous output
print("Max difference between dot product and matrix multiplication versions:", 
      np.abs(conv_outputs - conv_outputs_mat).max())

# Compare with the provided reference output
conv_outputs_flat = conv_outputs.reshape((n_p, nf, n), order='C')
print("Max difference with reference output:", 
      np.abs(conv_outputs_mat - conv_outputs_flat).max())

Max difference between dot product and matrix multiplication versions: 1.7763568394002505e-15
Max difference with reference output: 1.7763568394002505e-15


## Step 3 - Einstein Summation

In [13]:
# Compute convolution using einsum
conv_outputs_einsum = np.einsum('ijn,jl->iln', MX, Fs_flat, optimize=True)

# Compare with matrix multiplication version
print("Max difference between matrix multiplication and einsum versions:", 
      np.abs(conv_outputs_mat - conv_outputs_einsum).max())

# Compare with the provided reference output
print("Max difference between einsum and reference output:", 
      np.abs(conv_outputs_einsum - conv_outputs_flat).max())

Max difference between matrix multiplication and einsum versions: 0.0
Max difference between einsum and reference output: 1.7763568394002505e-15


## To compare which is fastest

In [14]:
def time_method(method, runs=5):
    times = []
    for _ in range(runs):
        start = time.time()
        if method == "dot":
            # Dot product version
            conv_out = np.zeros((n_p, nf, n))
            for i in range(n):
                for j in range(nf):
                    patch_idx = 0
                    for h in range(0, 32, f):
                        for w in range(0, 32, f):
                            patch = X_ims[h:h+f, w:w+f, :, i]
                            filter = Fs[:, :, :, j]
                            conv_out[patch_idx, j, i] = np.multiply(patch, filter).sum()
                            patch_idx += 1
        elif method == "matmul":
            # Matrix multiplication version
            conv_out = np.zeros((n_p, nf, n))
            for i in range(n):
                conv_out[:, :, i] = np.matmul(MX[:, :, i], Fs_flat)
        else:
            # Einsum version
            conv_out = np.einsum('ijn,jl->iln', MX, Fs_flat, optimize=True)
        times.append(time.time() - start)
    return np.mean(times), np.std(times)


In [15]:

methods = ["dot", "matmul", "einsum"]
for method in methods:
    mean_time, std_time = time_method(method)
    print(f"{method} method: {mean_time:.4e}s ± {std_time:.2e}s")

dot method: 1.8545e-02s ± 1.78e-02s
matmul method: 2.4986e-05s ± 5.23e-06s
einsum method: 9.5558e-05s ± 5.06e-05s


## Conclusion
It is well known that the einsum is much faster. It cannot be seen in this example, since we only work with 5 images. But it should be known that when using many images, einsum tends to be much faster.